# Advent of Code - 2021

## Daily Data

The input data for each day "e.g. day 5" should be downloaded into the data subdirectory for the corresponding day (e.g. data/day_5/input.txt). This will allow use of the `get_daily_data` generator function.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline

## Support Functions

In [127]:
import os
def get_daily_data(day, sep=None):
    daily_filename = "data/day_{}/input.txt".format(day)
    if not os.path.exists(daily_filename):
        try:
            os.rename("/Users/mikes/Downloads/input.txt", daily_filename)
        except:
            raise Exception("Could not find daily input file.")
    with open(daily_filename, 'r') as fin:
        for line in fin.readlines():
            if sep is None:
                yield line.rstrip()
            else:
                yield line.rstrip().split(sep)

## Day 4

In [220]:
class BingoBoard:
    
    def __init__(self, values):
        self.values = np.array(values)
        self.marked = np.zeros(self.values.shape).astype(bool)
        
    def mark(self, value):
        idx = np.where(self.values == value)
        self.marked[idx] = True
    
    def bingo(self):
        row_bingo = (self.marked.sum(axis=1) == self.marked.shape[0]).any()
        col_bingo = (self.marked.sum(axis=0) == self.marked.shape[1]).any()
        return row_bingo or col_bingo
        
    def get_unmarked_values(self):
        unmarked = ~self.marked
        return self.values[np.where(unmarked)]

In [221]:
def init_data():
    data = [line for line in get_daily_data(4)]
    balls = [int(x) for x in data.pop(0).split(',')]
    data.pop(0)  # throw away empty line

    boards = []
    board_values = []
    for line in data:
        if line == "":
            boards.append(BingoBoard(board_values))
            board_values = []
        else:
            row = [int(x) for x in line.split()]
            board_values.append(row)
    return boards, balls

In [222]:
def play_bingo(boards, balls):
    for bl in balls:
        for bd in boards:
            bd.mark(bl)
            if bd.bingo():
                solution = bd.get_unmarked_values().sum() * bl
                return solution

In [223]:
boards, balls = init_data()
play_bingo(boards, balls)

87456

In [224]:
def play_bad_bingo(boards, balls):
    for bl in balls:
        purge = []
        for bd in boards:
            bd.mark(bl)
            if bd.bingo():
                if len(boards) == 1:
                    solution = bd.get_unmarked_values().sum() * bl
                    return solution
                else:
                    purge.append(bd)
        for bd in purge:
            boards.remove(bd)

In [225]:
boards, balls = init_data()
play_bad_bingo(boards, balls)

15561

## Day 3

In [5]:
def to_decimal(binary_vector):
    total = 0
    for pos, bit in enumerate(binary_vector[::-1]):
        total += bit * (2 ** pos)
    return total

In [6]:
import numpy as np
from scipy import stats

data = []
for line in get_daily_data(3):
    data.append([int(x) for x in list(line.strip())])
data = np.array(data)

In [7]:
gamma_bin = list(stats.mode(data, axis=0)[0][0])
epsilon_bin = [abs(data - 1) for data in gamma_bin]
gamma = to_decimal(gamma_bin)
epsilon = to_decimal(epsilon_bin)
gamma * epsilon

2967914

In [8]:
def most_common(vec):
    n1 = sum(vec)
    n0 = len(vec) - sum(vec)
    if n0 == n1:
        return 1
    elif n0 > n1:
        return 0
    else:
        return 1

In [9]:
def least_common(vec):
    n1 = sum(vec)
    n0 = len(vec) - sum(vec)
    if n0 == n1:
        return 0
    elif n0 > n1:
        return 1
    else:
        return 0

In [10]:
def find_og_rating(data):
    for col in range(data.shape[1]):
        x = data[:, col]
        mode = most_common(x)
        data = data[x == mode, :]
        if data.shape[0] == 1:
            break
    assert(data.shape[0] == 1)
    return to_decimal(data[0, :])

In [11]:
def find_co2_rating(data):
    for col in range(data.shape[1]):
        x = data[:, col]
        antimode = least_common(x)
        data = data[x == antimode, :]
        if data.shape[0] == 1:
            break
    assert(data.shape[0] == 1)
    return to_decimal(data[0, :])

In [12]:
og = find_og_rating(data)
og

1927

In [13]:
co2 = find_co2_rating(data)
co2

3654

In [14]:
og * co2

7041258

In [15]:
# 7105014 is too high

## Day 2

In [16]:
horizontal_loc = 0
depth_loc = 0
for direction, value in get_daily_data(2, ' '):
    value = int(value)
    if direction == "forward":
        horizontal_loc += value
    elif direction == "down":
        depth_loc += value
    elif direction == "up":
        depth_loc -= value
horizontal_loc * depth_loc

1580000

In [17]:
horizontal_loc = 0
depth_loc = 0
aim = 0
for direction, value in get_daily_data(2, ' '):
    value = int(value)
    if direction == "forward":
        horizontal_loc += value
        depth_loc += value * aim
    elif direction == "down":
        aim += value
    elif direction == "up":
        aim -= value
horizontal_loc * depth_loc

1251263225

## Day 1

In [18]:
n_increases = 0
prev_x = 9e32
for value in get_daily_data(1):
        x = int(value)
        if x > prev_x:
            n_increases += 1
        prev_x = x
n_increases

1466

In [19]:
n_increases = 0
window = []
prev_sum = None
for value in get_daily_data(1):
    x = int(value)
    if len(window) < 3:
        window.append(x)
    if prev_sum is None:
        prev_sum = sum(window)
    if len(window) == 3:
        window.pop(0)
        window.append(x)
        new_sum = sum(window)
        if new_sum > prev_sum:
            n_increases += 1
        prev_sum = new_sum
n_increases

1491